In [1]:
import requests
from requests.auth import HTTPBasicAuth
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import re
import uuid 
import json
import sqlite3

pd.set_option('display.max_rows', 900)

In [2]:
class BETS():
    def __init__(self):
        load_dotenv()
        self.clientId = os.getenv('CLIENT_ID')
        self.clientPw = os.getenv('CLIENT_PW')
        self.baseUrl = "https://api.tournamentsoftware.com/1.0"
    
    def make_call(self,url):
        response = requests.get(url, auth=HTTPBasicAuth(self.clientId, self.clientPw))
        return response

    #   USE SEARCH QUERY '?q=U11 Gold'
    #   Filter by dates
    #   Create lists of cool tourney codes.
    def search_tournaments(self,dateStr):
        splitDate = dateStr.split('/')
        refDate = splitDate[2] + '-' + splitDate[1] + '-' + splitDate[0]
        url = self.baseUrl+f"/Tournament?list=1&refdate={refDate}&pagesize=1000" 
        return self.make_call(url)
    
    def get_tournament_details(self,tCode):
        url = self.baseUrl+f"/Tournament/{tCode}"
        return self.make_call(url)

    def get_matches(self,tCode,date):
        dateQuery = date.strftime("%Y%m%d")
        url = self.baseUrl+f"/Tournament/{tCode}/Match/{dateQuery}"
        return self.make_call(url)
    
    def wanted_rounds(self,roundCaredAbout):
        allRounds = ['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'Final']
        if roundCaredAbout == 'Winner':
            return ['Final']
        elif roundCaredAbout in allRounds:
            index = allRounds.index(roundCaredAbout)
            return allRounds[index:]
        else:
            return ['','','','','','','']


In [3]:
def processXMLList(list):
    out = []
    for xml in list:
        out.append(xml.text)
    return out

def get_exclude_list():
    tabuTitles = ['invite','(1000ie)','masters','para','yonex','league','regional','schools','graded','rising','handicap']
    return tabuTitles

def get_tourney_deets(title):
    title = title.lower()
    age = ''
    level = ''

    if 'tier 4' in title:
        level = 'Tier 4'
    if 'bronze' in title:
        level = 'Bronze'
    if 'silver' in title:
        level = 'Silver'
    if 'gold' in title:
        level = 'Gold'
    if 'national' in title:
        level = 'Nationals'
    if 'restricted' in title:
        level = 'Restricted'
    
    ageGroupRegex = re.compile('u(1[0-9])')
    ageMatch = re.search(ageGroupRegex,title)
    if ageMatch:
        age = ageMatch.group().upper()
    elif 'senior' in title:
        age = 'Senior'

    return age,level



#Both inputs as strings, searchStart = d/m/y, searchText = 'U11 Gold'
def get_tournament_results(bets,searchStartStr):
    tabuList = get_exclude_list()
    searchStart = datetime.strptime(searchStartStr,"%d/%m/%Y")
    tourneysDf = pd.DataFrame(columns=['Name','Code','StartDate','EndDate'])
    tourneysToAdd = []

    out = bets.search_tournaments(searchStartStr).content
    soup = BeautifulSoup(out,'xml')
    tournaments = soup.find_all('Tournament')
    for i in range(0,len(tournaments)):  
        name = tournaments[i].find('Name').text
        code = tournaments[i].find('Code').text
        startDate = datetime.fromisoformat(tournaments[i].find('StartDate').text)
        endDate = datetime.fromisoformat(tournaments[i].find('EndDate').text)
        tType = tournaments[i].find('TypeID').text
        
        goodTourney = True
        if not tType == '0':
            goodTourney = False
        
        for t in tabuList:
            if t in name.lower():
                goodTourney = False
        if not any(existing_tourney['Name'] == name for existing_tourney in tourneysToAdd) and goodTourney:
            tourney = {
                "Name": name,
                "Code": code,
                "StartDate": startDate,
                "EndDate": endDate,
            }

            tourneysToAdd.append(tourney)

    tourneysDf = pd.concat([tourneysDf, pd.DataFrame(tourneysToAdd)], ignore_index=True)
    searchUntil = datetime.today() - timedelta(days = 2)
    searchFrom = searchStart

    
    # Apply condition and filter rows
    tourneysInRangeDf = tourneysDf[(tourneysDf['StartDate'] <= searchUntil)].copy()

    linksDf = tourneysInRangeDf[['Name','Code','StartDate']]

    def code_to_link(code):
        return f'https://be.tournamentsoftware.com/tournament/{code}'
    
    linksDf['Code'] = linksDf['Code'].apply(code_to_link)


    #Itterate through Tournaments
    match_data = []
    for index, row in tourneysInRangeDf.iterrows():
        dtCurrentDay = row['StartDate']
        dtEndDate = row['EndDate']
        code = row['Code']
        tName = row['Name']
        

        
        #Itterate through Days With Matches.
        allMatchesXML = [] 
        while dtCurrentDay <= dtEndDate:
            tDetails = bets.get_matches(row['Code'],dtCurrentDay)
            matchesXML = tDetails.content
            allMatchesXML.append(matchesXML)
            dtCurrentDay = dtCurrentDay + timedelta(days=1)

        strCurrentDay = datetime.strftime(dtCurrentDay,'%d/%m/%Y')
        tAge,tLevel = get_tourney_deets(tName)
        # Iterate over each Match element
        for matchesXML in allMatchesXML:
            soup = BeautifulSoup(matchesXML,'xml')
            for match in soup.find_all('Match'):

                #check if match went ahead:
                if not match.find('ScoreStatus').text == '0':
                    pass
                else:
                    try:
                        roundName = match.find('RoundName').text
                    except:
                        roundName = 'Not Found'
                        print('no round name for match:',match)
                    match_info = {
                        'Tournament': tName,
                        'Date': strCurrentDay,
                        'Age': tAge,
                        'Level':tLevel,
                        'Code': match.find('Code').text,
                        'Winner': match.find('Winner').text,
                        'ScoreStatus': match.find('ScoreStatus').text,
                        'RoundName': roundName,
                        'EventName': match.find('EventName').text,
                    }
                    # Extract team and player details
                    teams = []
                    for team in ['Team1', 'Team2']:
                        teamXML = match.find(team)
                        team_info = {}
                        #find player id's and if singles
                        team_info[f'{team}_MemberID'] = processXMLList(teamXML.find_all('MemberID'))
                        team_info[f'{team}_Firstname'] = processXMLList(teamXML.find_all('Firstname'))
                        team_info[f'{team}_Lastname'] = processXMLList(teamXML.find_all('Lastname'))
                        team_info[f'{team}_GenderID'] = processXMLList(teamXML.find_all('GenderID'))

                        teams.append(team_info)

                    match_info.update(teams[0])
                    match_info.update(teams[1])

                    # Extract sets scores
                    sets = match.find('Sets')
                    set_scores = []
                    if sets:
                        for set_ in sets.find_all('Set'):
                            set_scores.append({
                                'Set_Team1': set_['Team1'],
                                'Set_Team2': set_['Team2']
                            })
    
                        match_info['Set_Scores'] = set_scores
                    

                    # Append the structured match information to the list
                    match_data.append(match_info)

            # Convert the list to a DataFrame
    df = pd.DataFrame(match_data)
    return df,linksDf


In [4]:
bets = BETS()
matchData,linksDf = get_tournament_results(bets,'1/12/2024') #d m y

/tmp/ipykernel_11903/2093604030.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linksDf['Code'] = linksDf['Code'].apply(code_to_link)


In [5]:
matchData.head(1)

,Tournament,Date,Age,Level,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,Team1_Firstname,Team1_Lastname,Team1_GenderID,Team2_MemberID,Team2_Firstname,Team2_Lastname,Team2_GenderID,Set_Scores
0,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,175,2,0,R1,Open Singles,[1358872],[Punit],[Kishorekumar],[1],[1396470],[Rhys Andrew],[Williams],[1],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set..."


In [6]:
# Format names
for team in ['Team1','Team2']:
    namesFormatted = []
    for index, row in matchData.iterrows():
        firstNames = row[f'{team}_Firstname']
        secondNames = row[f'{team}_Lastname']

        if not (len(firstNames) == len(secondNames)):
            raise ValueError(f'There are not matching numbers of first and second names when formatting.\n See below for row \n {row}')

        playerNames = []
        if len (firstNames) > 1: #Doubles
            
            for i in range(0,len(firstNames)):
                playerNames.append(firstNames[i] + ' ' +  secondNames[i])

        else: #Singles
            playerNames.append(firstNames[0] + ' ' +  secondNames[0])

        namesFormatted.append(playerNames)

    matchData[f'{team}_Names'] = namesFormatted


## Drop the original columns
#df = df.drop(['Col1', 'Col2'], axis=1)
#
#print(df)



In [7]:
#Format Event
matchData['Event'] = ''
matchData['Star'] = ''

for index,row in matchData.iterrows():
    strEvent = row['EventName'].lower().strip()
    print(strEvent)
 
    allEvents = ['os','od',
                 'ms','md','bs','bd',
                 'ws','wd','gs','gd',
                 'xd'] 

    eventOut = ''

    isStar = False
    if '*' in strEvent: # Check if its a star event
        isStar = True

    if (strEvent[:2] in allEvents): # handle two letter abreviation
        eventOut = strEvent[:2].upper().replace('M','O').replace('G','W').replace('B','O') #replace men, boys with open, girl with women.
    


    else: #Handle inputs such as 'open singles'
        if 'single' in strEvent:
            if ('men' in strEvent) or ('open' in strEvent) or ('boy' in strEvent):
                eventOut = 'OS'
            else:
                eventOut = 'WS'

        elif 'mix' in strEvent:
            eventOut = 'XD'

        elif 'double' in strEvent:
            if ('men' in strEvent) or ('open' in strEvent) or ('boy' in strEvent):
                eventOut = 'OD'
            else:
                eventOut = 'WD'
        

    if eventOut: # Append the abreviated event
        row['Event'] = eventOut 

        if isStar: #Add star flag if it was
            row['Star'] = True
        else:
            row['Star'] = False

    else:
        print('No known event for event:',row['EventName'])



#Format Gender
def replace_gender(value):
    if isinstance(value, list):  # Check if the entry is a list
        return ['M' if v == '1' else 'F' for v in value]
    return 'M' if value == '1' else 'F'

def replace_round(value):
    if 'R' in value:
        roundNum = value[1:]
        if roundNum in ['16','32','128','256']:
            return 'Round Of ' + roundNum
        return 'Round ' + value[1:]
    if value == 'SF':
        return 'Semi-Final'
    if value == 'QF':
        return 'Quarter-Final'
    if value == 'Final':
        return value

# Apply the replacement
for team in ['Team1','Team2']:
    matchData[f'{team}_Gender'] = matchData[f'{team}_GenderID'].apply(replace_gender)

# Create list of all ID's for quick indexing
allIds = []
for index,row in matchData.iterrows():
    rowIds = []
    for team in ['Team1','Team2']:
        for id in row[f'{team}_MemberID']:
            rowIds.append(id)
    allIds.append(rowIds)
matchData['PID'] = allIds

matchData['Round'] = matchData['RoundName'].apply(replace_round)

matchData#.head(5)


open singles
open singles
open singles
open singles
open singles
open singles
open singles
open singles
gs
gs
gs
gs
gs
gs
gs
open singles
open singles
open singles
open singles
open singles
open singles
open singles
open singles
open singles*
gs
open singles*
gs
gs
open singles*
gs
open singles*
gs*
gs
gs*
gs*
gs*
gs
open singles
open singles
open singles
open singles
open singles
open singles
open singles
gs
open singles*
open singles*
gs
gs
open singles*
gs
gs
gs
gs*
gs*
gs*
gs*
open singles
open singles*
open singles
open singles*
open singles
open singles
open singles*
gs
gs*
gs*
gs*
gs
open doubles
open doubles
open doubles
open doubles
open doubles
gs
gd
gd
gd
gd
gd
open doubles
open doubles
open doubles
open doubles
open doubles
open doubles
gd
gd
gd
gd
gd
open doubles
open doubles
open doubles
open doubles
open doubles
gd
gd
gd
gd
gd
open doubles
open doubles
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
open singles*
open singles*
gs
gs*
gs
gs*


,Tournament,Date,Age,Level,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,...,Team2_GenderID,Set_Scores,Team1_Names,Team2_Names,Event,Star,Team1_Gender,Team2_Gender,PID,Round
0,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,175,2,0,R1,Open Singles,[1358872],...,[1],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set...",[Punit Kishorekumar],[Rhys Andrew Williams],OS,False,[M],[M],"[1358872, 1396470]",Round 1
1,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,184,1,0,R1,Open Singles,[1298311],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '14'}, {'Set...",[Shaurya Singh],[Ayush Iyengar],OS,False,[M],[M],"[1298311, 1350351]",Round 1
2,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,193,1,0,R1,Open Singles,[1346258],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '12'}, {'Set...",[Kevin Tao],[Anish Reddy Kokatam],OS,False,[M],[M],"[1346258, 1379398]",Round 1
3,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,139,1,0,R1,Open Singles,[1370416],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '12'}, {'Set...",[Suchir Krishna Addagondla],[Fausto Ku],OS,False,[M],[M],"[1370416, 1343190]",Round 1
4,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,202,1,0,R1,Open Singles,[1360210],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '17'}, {'Set...",[Suhas Govindarajula],[Ethan Feakin],OS,False,[M],[M],"[1360210, 1340995]",Round 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,2025 Northumberland Senior Gold,06/01/2025,Senior,Gold,40,1,0,R16,OS Senior,[1209602],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '8'}, {'Set_...",[Philip Pahatouridis],[Imran Wadud],OS,False,[M],[M],"[1209602, 1281522]",Round Of 16
2406,2025 Northumberland Senior Gold,06/01/2025,Senior,Gold,97,1,0,QF,WD Senior,"[1261726, 1236744]",...,"[2, 2]","[{'Set_Team1': '21', 'Set_Team2': '17'}, {'Set...","[Jessica Allen, Abbie Smith]","[Jody Barral, Jennifer Lafferty]",WD,False,"[F, F]","[F, F]","[1261726, 1236744, 1368735, 1280836]",Quarter-Final
2407,2025 Northumberland Senior Gold,06/01/2025,Senior,Gold,57,2,0,R32,OS Senior,[1180297],...,[1],"[{'Set_Team1': '19', 'Set_Team2': '21'}, {'Set...",[Nathan Rossiter],[Oliver Wu],OS,False,[M],[M],"[1180297, 1268282]",Round Of 32
2408,2025 Northumberland Senior Gold,06/01/2025,Senior,Gold,37,2,0,QF,OS Senior,[1255662],...,[1],"[{'Set_Team1': '11', 'Set_Team2': '21'}, {'Set...",[Oliver Nicolson],[Jack Taylor],OS,False,[M],[M],"[1255662, 1177391]",Quarter-Final


In [8]:
explosionTest = matchData.explode('PID')

In [9]:
#For later: 'Tournament','Date','Age','Level', 'RoundName'
wantedCols = ['PID','Tournament','Date','Age','Level','Event','Round','Winner','Team1_MemberID','Team1_Names','Team1_Gender','Team2_MemberID','Team2_Names','Team2_Gender','Set_Scores']

explosionTest = explosionTest[wantedCols]
explosionTest

,PID,Tournament,Date,Age,Level,Event,Round,Winner,Team1_MemberID,Team1_Names,Team1_Gender,Team2_MemberID,Team2_Names,Team2_Gender,Set_Scores
0,1358872,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,[1358872],[Punit Kishorekumar],[M],[1396470],[Rhys Andrew Williams],[M],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set..."
0,1396470,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,[1358872],[Punit Kishorekumar],[M],[1396470],[Rhys Andrew Williams],[M],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set..."
1,1298311,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,[1298311],[Shaurya Singh],[M],[1350351],[Ayush Iyengar],[M],"[{'Set_Team1': '21', 'Set_Team2': '14'}, {'Set..."
1,1350351,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,[1298311],[Shaurya Singh],[M],[1350351],[Ayush Iyengar],[M],"[{'Set_Team1': '21', 'Set_Team2': '14'}, {'Set..."
2,1346258,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,[1346258],[Kevin Tao],[M],[1379398],[Anish Reddy Kokatam],[M],"[{'Set_Team1': '21', 'Set_Team2': '12'}, {'Set..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,1177391,2025 Northumberland Senior Gold,06/01/2025,Senior,Gold,OS,Quarter-Final,2,[1255662],[Oliver Nicolson],[M],[1177391],[Jack Taylor],[M],"[{'Set_Team1': '11', 'Set_Team2': '21'}, {'Set..."
2409,1137823,2025 Northumberland Senior Gold,06/01/2025,Senior,Gold,WD,Quarter-Final,2,"[1137823, 1191192]","[Sophie Brealey, Rachel Vickers]","[F, F]","[1309991, 1271586]","[Lucy Dodd, Sophie Watson]","[F, F]","[{'Set_Team1': '16', 'Set_Team2': '21'}, {'Set..."
2409,1191192,2025 Northumberland Senior Gold,06/01/2025,Senior,Gold,WD,Quarter-Final,2,"[1137823, 1191192]","[Sophie Brealey, Rachel Vickers]","[F, F]","[1309991, 1271586]","[Lucy Dodd, Sophie Watson]","[F, F]","[{'Set_Team1': '16', 'Set_Team2': '21'}, {'Set..."
2409,1309991,2025 Northumberland Senior Gold,06/01/2025,Senior,Gold,WD,Quarter-Final,2,"[1137823, 1191192]","[Sophie Brealey, Rachel Vickers]","[F, F]","[1309991, 1271586]","[Lucy Dodd, Sophie Watson]","[F, F]","[{'Set_Team1': '16', 'Set_Team2': '21'}, {'Set..."


In [10]:
def make_player_in_team1(row):
    if str(row['PID']) in row['Team2_MemberID']:  # If the player is in Team2, swap
        # Swap all relevant columns
        row['Team1_MemberID'], row['Team2_MemberID'] = row['Team2_MemberID'], row['Team1_MemberID']
        row['Team1_Names'], row['Team2_Names'] = row['Team2_Names'], row['Team1_Names']
        row['Team1_Gender'], row['Team2_Gender'] = row['Team2_Gender'], row['Team1_Gender']
        
        # Swap set scores (Team1 ↔ Team2)
        if isinstance(row['Set_Scores'], list):
            swapped_scores = []
            for score in row['Set_Scores']:
                # Ensure 'Set_Team1' and 'Set_Team2' keys exist before swapping
                swapped_scores.append({'Set_Team1': score['Set_Team2'], 'Set_Team2': score['Set_Team1']})
            row['Set_Scores'] = swapped_scores
        
        # Update winner: If originally Team2 won (Winner = 2), set Winner = 1, and vice versa
        prevWinner = int(row['Winner'])
        row['Winner'] = 1 if prevWinner == 2 else 2

    return row

# Apply the function to each row
preCorrectedDf = explosionTest.copy()
correctedDf = explosionTest.apply(make_player_in_team1, axis=1)


### The following is for database dramas

In [11]:
# Copy DataFrame to avoid modifying the original
df = explosionTest.copy()
db_name = 'local_database.db'
# Function to generate unique MatchID
def generate_match_id(row):
    date = row['Date'].replace('/','')
    
    # Get first player ID from Team1_MemberID (assuming it's a list)
    first_team1_id = str(row['PID'])
    
    # Convert Team2_MemberID to a string (flatten if it's a list)
    team2_id = ''.join(map(str, row['Team2_MemberID'])) if isinstance(row['Team2_MemberID'], list) else str(row['Team2_MemberID'])
    
    event = row['Event']
    
    # Concatenate all parts into a unique string
    return f"{date}_{first_team1_id}_{team2_id}_{event}"

# Apply function to create MatchID column
df.insert(0, 'MatchID', df.apply(generate_match_id, axis=1))

for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, (list, dict))).any():
        df[col] = df[col].apply(json.dumps) 

def save_to_database(df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Fetch existing MatchIDs from the database
    query = f"SELECT MatchID FROM {table_name}"
    try:
        existing_match_ids = pd.read_sql(query, conn)["MatchID"].tolist()
    except Exception as e:
        print(f"Table '{table_name}' does not exist or no MatchID column found. Proceeding with full insert.")
        existing_match_ids = []

    # Filter out matches that already exist
    df_filtered = df[~df["MatchID"].isin(existing_match_ids)]

    if not df_filtered.empty:
        df_filtered.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Added {len(df_filtered)} new matches to '{table_name}' in '{db_name}'.")
    else:
        print("No new matches to add.")

    conn.close()

# Save DataFrame to the database
save_to_database(df, db_name, table_name="your_table")





Added 315 new matches to 'your_table' in 'local_database.db'.


In [12]:
#Cull matches from db which are older than days_threshold days old.
def remove_old_matches(db_name, table_name, days_threshold=850):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Compute the threshold date
    threshold_date = (datetime.now() - timedelta(days=days_threshold)).strftime("%Y-%m-%d")  # Convert to YYYY-MM-DD

    # Convert Date column to proper format and delete old matches
    delete_query = f"""
    DELETE FROM {table_name} 
    WHERE DATE(substr(Date, 7, 4) || '-' || substr(Date, 4, 2) || '-' || substr(Date, 1, 2)) < ?
    """
    cursor.execute(delete_query, (threshold_date,))

    # Commit changes and close connection
    conn.commit()
    deleted_rows = cursor.rowcount
    conn.close()

    threshold_date = datetime.strptime(threshold_date,"%Y-%m-%d")
    threshold_date = datetime.strftime(threshold_date,'%d/%m/%Y')
    print(f"Removed {deleted_rows} old matches (before {threshold_date}) from '{table_name}'.")

# Example Usage:
# Remove matches older than 30 days
remove_old_matches(db_name, table_name="your_table", days_threshold=15)


Removed 2773 old matches (before 17/01/2025) from 'your_table'.


In [13]:
# Load the DataFrame from SQLite
conn = sqlite3.connect(db_name)
df_loaded = pd.read_sql(f"SELECT * FROM your_table", conn)

## Convert JSON strings back to lists/dicts
#for col in df_loaded.columns:
#    if df_loaded[col].apply(lambda x: isinstance(x, str) and x.startswith("[") or x.startswith("{") ).any():
#        df_loaded[col] = df_loaded[col].apply(json.loads)


conn.close()

df_loaded

,MatchID,PID,Tournament,Date,Age,Level,Event,Round,Winner,Team1_MemberID,Team1_Names,Team1_Gender,Team2_MemberID,Team2_Names,Team2_Gender,Set_Scores
0,27012025_1370416_1343190_OS,1370416,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,"[""1370416""]","[""Suchir Krishna Addagondla""]","[""M""]","[""1343190""]","[""Fausto Ku""]","[""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""12""}, {""Set..."
1,27012025_1343190_1370416_OS,1343190,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,"[""1343190""]","[""Fausto Ku""]","[""M""]","[""1370416""]","[""Suchir Krishna Addagondla""]","[""M""]","[{""Set_Team1"": ""12"", ""Set_Team2"": ""21""}, {""Set..."
2,27012025_1352848_1334745_OS,1352848,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,"[""1352848""]","[""Navaj Nandwani""]","[""M""]","[""1334745""]","[""Michael Williams""]","[""M""]","[{""Set_Team1"": ""18"", ""Set_Team2"": ""21""}, {""Set..."
3,27012025_1334745_1352848_OS,1334745,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,"[""1334745""]","[""Michael Williams""]","[""M""]","[""1352848""]","[""Navaj Nandwani""]","[""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""18""}, {""Set..."
4,27012025_1294628_1328905_OS,1294628,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,"[""1294628""]","[""Luke Parkinson""]","[""M""]","[""1328905""]","[""Yusuf Bilal""]","[""M""]","[{""Set_Team1"": ""18"", ""Set_Team2"": ""21""}, {""Set..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17615,19012025_1365296_13484381374085_OD,1365296,Swindon Stars U17 Bronze Jan 2025,19/01/2025,U17,Bronze,OD,Semi-Final,2,"[""1339142"", ""1365296""]","[""Max Cundall"", ""Tom Randall-Coles""]","[""M"", ""M""]","[""1348438"", ""1374085""]","[""Yeoh Chee Keong"", ""Ahmed Shah""]","[""M"", ""M""]","[{""Set_Team1"": ""20"", ""Set_Team2"": ""22""}]"
17616,19012025_1351728_13484381374085_OD,1351728,Swindon Stars U17 Bronze Jan 2025,19/01/2025,U17,Bronze,OD,Final,1,"[""1351728"", ""1346056""]","[""Etienne Fan"", ""Eden Gough""]","[""M"", ""M""]","[""1348438"", ""1374085""]","[""Yeoh Chee Keong"", ""Ahmed Shah""]","[""M"", ""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""17""}]"
17617,19012025_1346056_13484381374085_OD,1346056,Swindon Stars U17 Bronze Jan 2025,19/01/2025,U17,Bronze,OD,Final,1,"[""1351728"", ""1346056""]","[""Etienne Fan"", ""Eden Gough""]","[""M"", ""M""]","[""1348438"", ""1374085""]","[""Yeoh Chee Keong"", ""Ahmed Shah""]","[""M"", ""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""17""}]"
17618,19012025_1348438_13517281346056_OD,1348438,Swindon Stars U17 Bronze Jan 2025,19/01/2025,U17,Bronze,OD,Final,2,"[""1348438"", ""1374085""]","[""Yeoh Chee Keong"", ""Ahmed Shah""]","[""M"", ""M""]","[""1351728"", ""1346056""]","[""Etienne Fan"", ""Eden Gough""]","[""M"", ""M""]","[{""Set_Team1"": ""17"", ""Set_Team2"": ""21""}]"



**A list of available information for each match:**
- `MatchID` : Unique identifier for each match - in the form: Date_PID_OppID(s)_Event.
- `PID` : The players BE ID
- `Tournament` : Name of the tournament.
- `Date` : Match date (format: DD/MM/YYYY).
- `Age` : Age of tournament.
- `Level` : The level of the tournament (bronze, silver, ect...).
- `Event` : Dicipline: OS, WS, OD, WD or XD.
- `Round` : Round of the tournament played.
- `Winner` : The winning team. Matches formatted so the PID of the match row is always in team 1.
- `TeamX_MemberID` : IDs of Team X members.
- `TeamX_Names` : Names of Team X players.
- `TeamX_Gender` : Gender of Team X players
- `Set_Scores` : Scores for each set played. stored as a dict. see below for more.


In [18]:
roundCols = explosionTest['Round']
roundCols.unique()

array(['Round 1', 'Round 2', 'Round 3', 'Quarter-Final', 'Semi-Final',
       'Final', 'Round Of 32', 'Round Of 16', 'Round 4', 'Round 5',
       'Round 6', 'Round 7'], dtype=object)

## Below is an example row

| MatchID                          | PID     | Tournament                               | Date       | Age  | Level | Event | Round   | Winner | Team1_MemberID | Team1_Names                   | Team1_Gender | Team2_MemberID | Team2_Names   | Team2_Gender | Set_Scores                                          |
|----------------------------------|---------|------------------------------------------|------------|------|-------|-------|---------|--------|----------------|--------------------------------|--------------|----------------|--------------|--------------|----------------------------------------------------|
| 27012025_1370416_1343190_OS     | 1370416 | U17 Gold Milton Keynes January 2025     | 27/01/2025 | U17  | Gold  | OS    | Round 1 | 1      | ["1370416"]    | ["Suchir Krishna Addagondla"] | ["M"]        | ["1343190"]    | ["Fausto Ku"] | ["M"]        | [{"Set_Team1": "21", "Set_Team2": "12"}, {"Set_Team1": "21", "Set_Team2": "14"}] |

### Formats

- Date: Always dd/mm/YY - Zero padded. 20/01/2024 

- Age: U{age} OR Senior - U9 -> U19, Senior

- Level: Tier 4, Bronze, Silver, Gold, Nationals, Restricted.

- Event: OS, WS, OD, WD, XD. all 'ms' or 'md' converts to Open

- Round: Round X, Round Of {16, 32}, Quarter-Final, Semi-Final, Final

- TeamX_Info: List of Strings
    - Names: [Name 1, Name 2]
    - Gender: ['M', 'F']
    - MemberID: [ID 1, ID 2]

- Set Scores: List of dict's for each set. [{"Set_Team1": "21", "Set_Team2": "12"}, {"Set_Team1": "21", "Set_Team2": "14"}]